# Pulse Sequencer Configuration using SCPI

## Overview
This document covers the basics of configuring a pulse sequencer using the SCPI interface of an OpenSync device. 

## Purpose
This example demonstrates how to communicate an OpenSync device using its SCPI interface and configure a pulse sequencer using the raw commands. It is assumed that you have some fundamental knowledge of how SCPI commands work. If not, then one can read the basics [here](). Additionally, OpenSync relies on a stable USB connection, so make sure one is present for this demonstration.

## Imports

In [1]:
from opensync import opensync

## Device Connection

In [2]:
avail_ports = opensync.device_comm_search()
avail_ports

['COM7']

## Device Information

In [3]:
with opensync.device_comm_managed(avail_ports[0]) as sync_device:
    print(opensync.device_system_version(sync_device))

['OpenPIV', 'OpenSync', '5A3F6CE6128EEB36', '0.2.0']


## Configuring a Pulse Sequencer
A pulse sequencer on an OpenSync device outputs a pulse sequencer through eight (8) output channels. It is only concerned with producing an output sequence once an internal trigger is recieved and not where that signal came from or its repitition frequency. A pulse sequencer synthesizes a pulse sequence through output states and delays to the next output state change. An output state is merely the bit pattern of high and low pins (e.g., the output channels) at a given time. The delay between output states is measured in the current pulse sequencer's data units. The data units are not really in physical units per se. For instance, setting the data units to `microsecond` and querrying the units would not return microseconds. Instead, it will return a number representing the unit conversion to the base time scale in nanoseconds (for microseconds, this is 1e3). The same thing can be said about the clock divider (DIVider command in SCPI). The timing resolution is given by multiplying the base time scale with the clock divider. So, setting the clock divider to `low_res` would return a value such as 25. It is important to keep these considerations in mind when configuring a pulse sequencer.

Now, lets say that we want to output a pulse sequence where channel 1 (index 0) is high for 2 seconds, channel 2 (index 1) is high for 2 seconds 1 second after channel 1 turn high, and channel 2 (index 2) turns high when channels 1 and 2 go low. The following pulse sequence will look like the following.
![Example Pulse Sequence](images/example_pulse_timing.png)

Since the sequencer changes output states per instruction, we can think of it as a snapshot of the pulse sequence at a given time. For every snapshot, the output state changes. Focusing on the output changes, we can see the there is an output state change at 0 seconds, 1 second, 2 seconds, 3 seconds, and 5 seconds. This will serve as our delay instructions when we get to it. Now, we can focus on what values the output state instructions need to be. The output states are stored as an integer representation of an 8 bit number. So, we can convert the output state changes to the following instructions:
 - '00000001' --> 1
 - '00000011' --> 3
 - '00000010' --> 2
 - '00000100' --> 4
 - '00000000' --> 0

We can now pair the output state changes with the delays to those output snapshots. Note, a delay of zero will set the instruction delay to the smallest supported instruction delay.
| Bit State | Int | Delay |
| --- | --- | -- |
| 00000001 | 1 | 1 |
| 00000011 | 3 | 1 |
| 00000010 | 2 | 1 |
| 00000100 | 4 | 2 |
| 00000000 | 0 | 0 |

Now that we got out instructions, we can send them to the OpenSync device. Instructions are not sent directly to the pulse sequencer. Instead, they are sent into a static cache buffer where they are stored, checked, and applied to a pulse sequencer. This is important to acknowledge since a pulse sequencer's instructions are not updated automatically.

### Load instructions to device

Since we are working on a time scale in seconds, we need to change the data units to seconds.

In [4]:
command_data_units = ':pulse0:units second'
command_data_unitsQ = ':pulse0:units?'

with opensync.device_comm_managed(avail_ports[0]) as sync_device:
    # Change data units to seconds for pulse sequencer 0
    resp = opensync.device_comm_write(
        sync_device,
        command_data_units
    )

    # Note, a blank response means a success.
    print(resp)

    # Querry the change
    resp = opensync.device_comm_write(
        sync_device,
        command_data_unitsQ
    )

    print(resp)

[]
['10.00000000000000e+08']


In [5]:
command_send_states = ':pulse:data:store:outputs 1,3,2,4,0'
command_send_delays = ':pulse:data:store:delays 1,1,1,2,0'
command_apply = ':pulse:data:store:apply 0' # Applies data to pulse sequencer 0

with opensync.device_comm_managed(avail_ports[0]) as sync_device:
    # Write output states to cache
    opensync.device_comm_write(
        sync_device,
        command_send_states
    )

    # Write delays for output states to cache
    opensync.device_comm_write(
        sync_device,
        command_send_delays
    )

    # Apply cache to pulse sequencer 0
    resp = opensync.device_comm_write(
        sync_device,
        command_apply
    )

    # Note, a blank response means a success.
    print(resp)

[]


Let's say that the pulse on channel 3 is to be 20 seconds long instead. If a delay is too long for the current clock resolution, the device will abort loading the parameters.

In [6]:
command_send_delays_new = ':pulse:data:store:delays 1,1,1,20,0'

with opensync.device_comm_managed(avail_ports[0]) as sync_device:
    # Write output states to cache
    opensync.device_comm_write(
        sync_device,
        command_send_states
    )

    # Write delays for output states to cache
    opensync.device_comm_write(
        sync_device,
        command_send_delays_new
    )

    # Apply cache to pulse sequencer 0
    resp = opensync.device_comm_write(
        sync_device,
        command_apply
    )

    # Note, a blank response means a success.
    print(resp)

['**ERROR: -222', ' "Data out of range"']


If we have output state delays of an unsupported length, we can change the clock divider so the clock resolution is lower. Here, we chose `med_res` which results in an 8 ns clock resolution.

In [7]:
command_clock_div = ':pulse0:divider med_res'

with opensync.device_comm_managed(avail_ports[0]) as sync_device:
    # Change data units to seconds for pulse sequencer 0
    resp = opensync.device_comm_write(
        sync_device,
        command_clock_div
    )
    
    print(resp)

[]


With the updated clock divider, we can now re-apply the new pulse instructions to sequencer 0.

In [8]:
with opensync.device_comm_managed(avail_ports[0]) as sync_device:
    # Write output states to cache
    opensync.device_comm_write(
        sync_device,
        command_send_states
    )

    # Write delays for output states to cache
    opensync.device_comm_write(
        sync_device,
        command_send_delays_new
    )

    # Apply cache to pulse sequencer 0
    resp = opensync.device_comm_write(
        sync_device,
        command_apply
    )

    # Note, a blank response means a success.
    print(resp)

[]


### Activating the pulse sequencer

Now that the pulse sequence has been loaded to the device, we can enable the pulse sequencer and select a pin number which triggers the pulse sequence to execute.

In [9]:
command_activate = ':pulse0:pin 0'

with opensync.device_comm_managed(avail_ports[0]) as sync_device:
    # Change pulse sequencer 0 signal pin to clock sequencer 0
    resp = opensync.device_comm_write(
        sync_device,
        command_clock_div
    )
    
    print(resp)

[]


In [10]:
command_activate = ':pulse0:state on'

with opensync.device_comm_managed(avail_ports[0]) as sync_device:
    # Change pulse sequencer 0 state to on
    resp = opensync.device_comm_write(
        sync_device,
        command_clock_div
    )
    
    print(resp)

[]


The pulse sequence will now be executed every time clock sequencer 0 sends a signal during program operation.